In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bz2
import random

In [23]:
# Load the TSV file into a DataFrame
info=pd.read_csv('C:/Users/Sara//Documents/Uni/Multimedia/id_information_mmsr.tsv', sep='\t')
tf_idf=pd.read_csv('C:/Users/Sara//Documents/Uni/Multimedia/id_lyrics_tf-idf_mmsr.tsv', sep='\t')

info
#tf_idf

,id,artist,song,album_name
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition)
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002)
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te
...,...,...,...,...
10090,zyzILCQvVeUFIINi,Crowded House,When You Come,Temple Of Low Men
10091,zzgS4ZqyswamEWNj,Britney Spears,My Only Wish (This Year),Platinum Christmas
10092,zzoFYDMlqU1X2zz1,Thundercat,DUI,Drunk
10093,zzpkRCGA5ud8q4mv,Otis Redding,Rock Me Baby,Otis Blue


In [24]:
# get song id for specific song
def get_song_id(song_name, artist_name):
    song_entry = info[(info['song'] == song_name) & (info['artist'] == artist_name)]
    
    if not song_entry.empty:
        song_id = song_entry.iloc[0]['id']
        return song_id
    else:
        return None  # Song not found


In [32]:
song_name = "Tra Te E Il Mare"
artist_name = "Laura Pausini"

song_id = get_song_id(song_name, artist_name)

if song_id is not None:
    print("Song ID for", song_name, "by", artist_name, "is:", song_id)
else:
    print("Song not found.")


Song ID for Tra Te E Il Mare by Laura Pausini is: 02ZnlCGZEbkfCDxo


In [33]:
# check if id is right
query_track_id = '02ZnlCGZEbkfCDxo'

track_info = info[info['id'] == query_track_id]
track_info

,id,artist,song,album_name
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te


In [34]:
#Task 1: Random Baseline

N = 10  # Define the number of tracks to retrieve

def retrieve_random_tracks(id, info, N):
    # Shuffle the songs DataFrame to get a random order
    shuffled_songs_df = info.sample(frac=1)
    
    # Exclude the query track using its ID
    shuffled_songs_df = shuffled_songs_df[shuffled_songs_df['id'] != query_track_id]
    
    # Select the top N rows as the retrieved tracks
    retrieved_tracks = shuffled_songs_df.head(N)
    
    return retrieved_tracks

# Example usage:
query_track_id = 123  # Replace with the actual query track ID
retrieved_tracks = retrieve_random_tracks(query_track_id, info, N)
print(retrieved_tracks)

                    id                 artist                      song  \
2469  EuFKcEtJ08PSZ3eH                    UGK                   One Day   
701   48mN3L6l1FWVFNUX  The Birthday Massacre                 Goodnight   
1239  7WzGPQNTJo6KhBVP                Rihanna                    Fading   
3412  Kzp3U8eHGgi3Xj2d           The Miracles              Love Machine   
8256  ohbvGoXAJ9BPZk51          Lindsay Lohan  Very Last Moment in Time   
6344  dN6dZS8y3P95uzVl                   FEMM            The Real Thing   
318   1gdgQLMaOVxRYj3e                Bodyjar              Not the Same   
8119  npNmFptTZAXcdEr1                   IAMX         I Like Pretending   
5486  XnfPW7kGspnUiKjT               CupcakKe            Meet and Greet   
1549  9COwMuVfKKA91Tzo            Smash Mouth          Come On, Come On   

                  album_name  
2469            Ridin' Dirty  
701   Walking With Strangers  
1239                    Loud  
3412          City Of Angels  
8256               

In [35]:
#Task 2: Text-based(cos-sim, tf-idf)
from sklearn.metrics.pairwise import cosine_similarity
# Locate the row corresponding to the query track in the DataFrame
query_track_row = tf_idf[tf_idf['id'] == song_id]
# Extract the TF-IDF values for the query track
query_track_tfidf = query_track_row.iloc[:, 2:]  # Assuming that columns from index 2 onwards contain TF-IDF values
query_track_tfidf





# Extract the relevant columns containing the TF-IDF values for the query vector
query_vector = query_track_row.iloc[:, 2:].values.reshape(1, -1)  # Assuming TF-IDF columns start from the third column

similarities = []

# Iterate through all tracks in the dataset
for index, row in tf_idf.iterrows():
    track_vector = row.iloc[2:].values.reshape(1, -1)  #start from third column
    similarity = cosine_similarity(query_vector, track_vector)[0][0]
    similarities.append((row['id'], similarity))

# Sort tracks by similarity
similarities.sort(key=lambda x: x[1], reverse=True)

# Retrieve the 10 most similar tracks
N = 10  # Number of similar tracks to retrieve
most_similar_tracks = similarities[1:N+1]

print("Most similar track IDs:", [song_id for song_id, similarity in most_similar_tracks])

Most similar track IDs: ['hyAOwBgnAOL7f66Q', 'hEBoVYWwigLAVxbJ', 'jZfcQW4j46BN4TBr', 'y5fJlGZks0kWeH5j', 'akpfejkAf7W9l0eO', 'BMaWLcICJh1Pj236', 'Nblp573S43B1TMzx', 'aPelENX8G49Gj6oj', 'uueOD75z68bYvRmH', 'gV4AH4V0cb6kk8j3']


In [36]:
def get_song_info_from_ids(track_ids, info_dataset):
    song_info = []
    for track_id in track_ids:
        info_entry = info_dataset[info_dataset['id'] == track_id]
        if not info_entry.empty:
            song_info.append((info_entry.iloc[0]['song'], info_entry.iloc[0]['artist']))
    return song_info


In [37]:
most_similar_track_ids = []

# Collect the IDs of the most similar tracks
for song_id, similarity in most_similar_tracks:
    most_similar_track_ids.append(song_id)

# Now you can use most_similar_track_ids as input to your get_song_info_from_ids function
# Example:
song_info = get_song_info_from_ids(most_similar_track_ids, info)

# Print the song and artist information
for song, artist in song_info:
    print(f"Song: {song}, Artist: {artist}")

Song: Entre Tu Y Mil Mares, Artist: Laura Pausini
Song: She Doesn't Live Here Anymore, Artist: Roxette
Song: Bien avant, Artist: Benjamin Biolay
Song: Don't Believe Anymore, Artist: Icehouse
Song: Valentine, Artist: The Get Up Kids
Song: I Won't Be Long, Artist: Beck
Song: Tu Quieres Volver, Artist: Gipsy Kings
Song: Long Way Down, Artist: Black Rebel Motorcycle Club
Song: Nevermore, Artist: Queen
Song: Go Robot, Artist: Red Hot Chili Peppers
